In [1]:
from datasets import load_dataset, concatenate_datasets
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load datasets
eng_sarc_dataset = load_dataset(path='raquiba/Sarcasm_News_Headline')
ar_sarc_dataset = load_dataset(path='ar_sarcasm', ignore_verifications=True)
kor_sarc_dataset = load_dataset(path='kor_sarcasm')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/raquiba--Sarcasm_News_Headline-2c0b20e2ac20e61f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2110 [00:00<?, ? examples/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/301 [00:00<?, ? examples/s]

Dataset kor_sarcasm downloaded and prepared to /root/.cache/huggingface/datasets/kor_sarcasm/default/1.1.0/00d38c200d4d563ed94efb9ff4ca119ded94fe3cdf1e381ed95274de0a9d59f0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
### process datasets
eng_sarc_dataset = eng_sarc_dataset.remove_columns(column_names=['article_link'])
eng_sarc_dataset = eng_sarc_dataset.rename_columns({"is_sarcastic":"label"})

ar_sarc_dataset = ar_sarc_dataset.remove_columns(column_names=['dialect', 'sentiment', 'original_sentiment', 'source'])
ar_sarc_dataset = ar_sarc_dataset.rename_columns({"tweet":"headline", "sarcasm":"label"})

kor_sarc_dataset = kor_sarc_dataset.rename_columns({"tokens":"headline"})

In [4]:
model_ckpt = 'distilbert-base-uncased'

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [6]:
# non_sarc_ar = ar_sarc_dataset.filter(lambda x: x['label'] == 0).select(range(2600))
# sarc_ar = ar_sarc_dataset.filter(lambda x: x['label'] == 1)
# ar_sarc_dataset = concatenate_datasets([sarc_ar, non_sarc_ar])

In [7]:
# tokenize datasets using distilbert pretrained tokenizer
encoded_eng_dataset = eng_sarc_dataset.map(lambda sample: tokenizer(sample['headline'], max_length=512, padding="max_length", truncation=True), batched=True).shuffle()
encoded_ar_dataset = ar_sarc_dataset.map(lambda sample: tokenizer(sample['headline'],  max_length=512, padding="max_length", truncation=True), batched=True).shuffle()
encoded_kor_dataset = kor_sarc_dataset.map(lambda sample: tokenizer(sample['headline'], max_length=512,  padding="max_length", truncation=True), batched=True).shuffle()

  0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
from transformers import AutoModelForSequenceClassification, DistilBertConfig
import torch 

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = DistilBertConfig(seq_classif_dropout=0.3)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.we

In [10]:
from transformers import TrainingArguments, Trainer

In [11]:
model_name = f"{model_ckpt}-multiling-sarcasm"
training_args = TrainingArguments(output_dir=model_name,
                                 num_train_epochs=3,
                                 learning_rate=2e-5,
                                 per_device_train_batch_size=32,
                                 per_device_eval_batch_size=32,
                                 evaluation_strategy='epoch',
                                 weight_decay=0.01,
                                 log_level='error',
                                 push_to_hub=False,
                                 report_to='none')

In [12]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np 

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {"f1": f1, "acc": accuracy}

In [13]:
# encoded_eng_dataset_rdy = encoded_eng_dataset.train_test_split(test_size=0.3)

In [14]:
encoded_eng_valid_test = encoded_eng_dataset['test'].train_test_split(test_size=0.2)
encoded_ar_valid_test = encoded_ar_dataset['test'].train_test_split(test_size=0.2)
encoded_kor_train_valid = encoded_kor_dataset['train'].train_test_split(test_size=0.1)

In [15]:
from datasets import DatasetDict

encoded_eng_splits = DatasetDict({
    "train": encoded_eng_dataset['train'],
    "valid": encoded_eng_valid_test['train'],
    "test": encoded_eng_valid_test['test']
})

encoded_ar_splits = DatasetDict({
    "train": encoded_ar_dataset['train'],
    "valid": encoded_ar_valid_test['train'],
    "test": encoded_ar_valid_test['test']
})

encoded_kor_splits = DatasetDict({
    "train": encoded_kor_train_valid['train'],
    "valid": encoded_kor_train_valid['test'],
    "test": encoded_kor_dataset['test']
})

In [16]:
trainer_eng = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_eng_splits['train'],
                 eval_dataset=encoded_eng_splits['valid'])
trainer_eng.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,No log,0.146489,0.945515,0.945430
2,0.281300,0.088188,0.973304,0.973323
3,0.143200,0.063288,0.980340,0.980344


TrainOutput(global_step=1344, training_loss=0.18271926187333606, metrics={'train_runtime': 2596.9777, 'train_samples_per_second': 33.06, 'train_steps_per_second': 0.518, 'total_flos': 1.1373253446408192e+16, 'train_loss': 0.18271926187333606, 'epoch': 3.0})

In [23]:
trainer_eng.predict(encoded_eng_splits['test'])

PredictionOutput(predictions=array([[ 1.6383013, -2.0405629],
       [ 2.6602166, -2.8882523],
       [-3.000528 ,  2.7888973],
       ...,
       [ 1.1908356, -1.1600237],
       [ 2.8151958, -3.1625926],
       [ 2.8945854, -3.2678792]], dtype=float32), label_ids=array([0, 0, 1, ..., 1, 0, 0]), metrics={'test_loss': 0.06209314987063408, 'test_f1': 0.979220810110723, 'test_acc': 0.9792212654436541, 'test_runtime': 50.1673, 'test_samples_per_second': 106.484, 'test_steps_per_second': 1.674})

In [24]:
trainer_ar = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_ar_splits['train'],
                 eval_dataset=encoded_ar_splits['valid'])
trainer_ar.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,No log,0.399300,0.770445,0.841232
2,No log,0.386956,0.770914,0.840047
3,No log,0.380160,0.779624,0.839455


TrainOutput(global_step=396, training_loss=0.3917424365727588, metrics={'train_runtime': 640.6067, 'train_samples_per_second': 39.511, 'train_steps_per_second': 0.618, 'total_flos': 3352882327382016.0, 'train_loss': 0.3917424365727588, 'epoch': 3.0})

In [ ]:
#encoded_kor_dataset_rdy = encoded_kor_dataset.train_test_split(test_size=0.25)

In [25]:
trainer_kor = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_kor_splits['train'],
                 eval_dataset=encoded_kor_splits['valid'])
trainer_kor.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,No log,0.517889,0.709644,0.716667
2,No log,0.488435,0.744830,0.745556
3,No log,0.482425,0.744141,0.744444


TrainOutput(global_step=381, training_loss=0.5235976572111836, metrics={'train_runtime': 593.2504, 'train_samples_per_second': 40.961, 'train_steps_per_second': 0.642, 'total_flos': 3218957787340800.0, 'train_loss': 0.5235976572111836, 'epoch': 3.0})

In [36]:
def concatenate_splits(corpora, cast_features):
    multi_corpus = DatasetDict()
    for split in corpora[0].keys():
        multi_corpus[split] = concatenate_datasets([corpus[split].cast(cast_features) for corpus in corpora]).shuffle()
    return multi_corpus        

In [34]:
eng_valid_test = eng_sarc_dataset['test'].train_test_split(test_size=0.2)
ar_valid_test = ar_sarc_dataset['test'].train_test_split(test_size=0.2)
kor_train_valid = kor_sarc_dataset['train'].train_test_split(test_size=0.1)

eng_splits = DatasetDict({
    "train": eng_sarc_dataset['train'],
    "valid": eng_valid_test['train'],
    "test": eng_valid_test['test']
})

ar_splits = DatasetDict({
    "train": ar_sarc_dataset['train'],
    "valid": ar_valid_test['train'],
    "test": ar_valid_test['test']
})

kor_splits = DatasetDict({
    "train": kor_train_valid['train'],
    "valid": kor_train_valid['test'],
    "test": kor_sarc_dataset['test']
})

In [39]:
multiling_dataset = concatenate_splits([eng_splits, ar_splits, kor_splits], cast_features=kor_splits['train'].features)

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
encoded_multiling_dataset = multiling_dataset.map(lambda sample: tokenizer(sample['headline'], padding='max_length', truncation=True, max_length=512), batched=True).shuffle()

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [43]:
encoded_multiling_dataset

DatasetDict({
    train: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 45156
    })
    valid: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 23955
    })
    test: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 6065
    })
})

In [44]:
trainer = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_multiling_dataset['train'],
                 eval_dataset=encoded_multiling_dataset['valid'])
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,0.227800,0.075625,0.971377,0.971405
2,0.197700,0.057869,0.977057,0.977082
3,0.148700,0.054467,0.978860,0.978877


TrainOutput(global_step=2118, training_loss=0.18451667709998976, metrics={'train_runtime': 3835.3652, 'train_samples_per_second': 35.321, 'train_steps_per_second': 0.552, 'total_flos': 1.7945093561131008e+16, 'train_loss': 0.18451667709998976, 'epoch': 3.0})